Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

__Julia Troni__

Note: Please first run the `TEST HELPER` cell that defines the `passed` function below. Failing to run this cell will make it hard for you to check your work.

In [18]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"Tests Passed (1 point)")
    else print(s"Tests Passed ($points points)") 
}

defined function passed

## Problem 1: CPS Style Interpreter For Lettuce (10 Points)

The class notes have a CPS style interpreter already for you.  You are welcome to understand how it is done there and write code here. We strongly discourage  cutting and pasting of code from notes to assignments but will not penalize students for doing that.

Here is a Grammar for Lettuce with some arithmetic, booleans, division, if-then-else and function calls.

$$\begin{array}{rcll}
  \mathbf{Expr} & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Div}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Geq}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{IfThenElse}(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{FunDef}(\mathbf{Ident}, \mathbf{Expr}) \\ 
  & | & \text{FunCall}(\mathbf{Expr}, \mathbf{Expr}) \\ 
  \end{array}$$
  
The scala definitions are given below for your convenience.

In [19]:
/*- 1. Abstract Syntax Tree for Expressions */
trait Expr
case class Const(d: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr 
case class Geq(e1: Expr, e2: Expr) extends Expr 
case class IfThenElse(eCond: Expr, eThen: Expr, eElse: Expr) extends Expr 
case class Let(id: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(param: String, body: Expr) extends Expr 
case class FunCall(e1: Expr, e2: Expr) extends Expr

/*- 2. Definitions of Environments and Values -*/
sealed trait Environment 
sealed trait Value

case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value 
case class Closure(param: String, expr: Expr, env: Environment ) extends Value

case object EmptyEnvironment extends Environment
case class Extend(s: String, v: Value, env: Environment) extends Environment
/* -- Define a runtime error exception --*/
case class RunTimeError(msg: String) extends Exception

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Div
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined trait Environment
defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object EmptyEnvironment
defined class Extend
defined class RunTimeError

Implement missing parts of some helper functions that perform the basic operations on values and environments in the CPS style below.

In [20]:
/* If v1 and v2 are NumValue, then add them, otherwise raise a 
   RunTimeError exception */
def addValue_k[T](v1: Value, v2: Value, k: Value => T ): T = (v1, v2) match {
    case (NumValue(d1), NumValue(d2)) => { 
        // YOUR CODE HERE
        k(NumValue(d1+d2))
    }
    case _ => {
        throw RunTimeError("Asked to add two values that are not numbers : type mismatch during runtime. ")
    }
}

/* Implement the function divValue using CPS style. However, if 
   the second value v2 is zero, then ensure that you are throwing 
   a RunTimeError exception */
def divValue_k[T](v1: Value, v2: Value, k: Value => T ): T = (v1, v2) match {
    case (NumValue(d1), NumValue(d2)) => { 
        // YOUR CODE HERE
        if (d2==0)  throw RunTimeError("Cannot divide by zero") 
        else k(NumValue(d1/d2))
    }
    case _ => {
        throw RunTimeError("Asked to divide two values that are not numbers : type mismatch during runtime. ")
    }
}

/* Implement >= comparison between numerical values. Values of any type other 
than numbers should raise a RunTimeError */
def geqValue_k[T](v1: Value, v2: Value, k: Value => T ): T = (v1, v2) match {
    case (NumValue(d1), NumValue(d2)) => { 
        // YOUR CODE HERE
        if (d1>=d2) {
            k(BoolValue(true))
        }
        else {
            k(BoolValue(false))
        }
    }
    case _ => {
        throw RunTimeError("Asked to compare two values that are not numbers : type mismatch during runtime. ")
    }
}

/* Lookup a identifierr if it is present in the environment, if not, raise a 
RuntimeError */
def lookup_k[T](env: Environment, x: String, k: Value => T): T = env match {
    case EmptyEnvironment => throw RunTimeError(s"Cannot find identifier: $x")
    case Extend(s, v, env2) if s == x => {
        // YOUR CODE HERE
        k(v)
    }
    case Extend(s, v, env2)  => {
        // YOUR CODE HERE
        k(lookup_k(env2,x, (v1: Value)=>v1))
    }
}



defined function addValue_k
defined function divValue_k
defined function geqValue_k
defined function lookup_k

In [21]:
def continue1(v: Value): String = v match {
    case NumValue(d) => d.toString
    case BoolValue(b) => b.toString
    case Closure(_,_,_) => "Some Closure"
}
def continue2(v: Value): Int = v match {
    case NumValue(_) => 1
    case BoolValue(_) => 2
    case Closure(_,_,_) => 3
}

val v1: String = addValue_k(NumValue(15), NumValue(30), continue1)
assert(v1 == "45.0", s"Test 1 Fail: expected 45.0 but got: $v1")

val v2: Int = addValue_k(NumValue(-15), NumValue(30), continue2)
assert(v2 == 1, s"Test 2 Fail: expected 1 but got: $v2")

val v3: String = divValue_k(NumValue(15), NumValue(30), continue1)
assert(v3 == "0.5", s"Test 3 Fail: expected 0.5 but got: $v3")

val v4: String = { 
    try {
     divValue_k(NumValue(15), NumValue(0), continue1);
     "NOT_OK"
    } catch {
        case RunTimeError(_) => "OK!!"
    }
}
assert(v4 == "OK!!", s"Test 4 Fail: expected OK but got: $v4")

val env = Extend("x", NumValue(10.0), 
                Extend("y", BoolValue(true), 
                    EmptyEnvironment ) )

val v5 = lookup_k(env, "y", continue2)
assert (v5 == 2, s"Test 5 Fail: expected 2 but got: $v5")
passed(5)

Tests Passed (5 points)

defined function continue1
defined function continue2
v1: String = "45.0"
v2: Int = 1
v3: String = "0.5"
v4: String = "OK!!"
env: Extend = Extend(
  s = "x",
  v = NumValue(d = 10.0),
  env = Extend(s = "y", v = BoolValue(b = true), env = EmptyEnvironment)
)
v5: Int = 2

Implement a CPS interpreter function `eval_k` using the helper functions defined above.

In [22]:
import scala.annotation.tailrec 

def eval_k[T](e: Expr, env: Environment, k: Value => T): T = e match {
    // YOUR CODE HERE
    
    case Const(f) => k(NumValue(f))

    case Ident(x) => {
        lookup_k(env,x,k )
    }
    
    case Plus(e1, e2) => {
       eval_k( e1, env, v1 => {
             eval_k(e2, env, v2 => {
                addValue_k(v1, v2, k)
          })
       })  
   }
    

    case Div(e1, e2) => {
        eval_k(e1,env, v1 => {
            eval_k(e2,env, v2 => {
                divValue_k(v1,v2,k)
            })
        })
    }
    
    case Geq(e1, e2) => {
        eval_k(e1,env, v1 => {
            eval_k(e2,env, v2 => {
                geqValue_k(v1,v2,k)
            })
        })
    }

    case IfThenElse(e1, e2, e3) => {
        eval_k(e1, env, {
            case BoolValue(true) => eval_k(e2, env, k )
            case BoolValue(false) => eval_k(e3, env,k )
            case _ => throw new IllegalArgumentException("If-then-else condition expr is non-boolean")
        })
    }

    case Let(x, e1, e2) => {
        eval_k(e1, env, v1 => {
                val env2 = Extend(x, v1, env) // create a new extended env
                eval_k(e2, env2, k) // eval e2 under that.
            })
    }

    case FunDef(x, e) => {
        k(Closure(x, e, env)) // Return a closure with the current enviroment.
    }

    case FunCall(e1, e2) => {
        eval_k(e1, env, {
                v1 => {
                    eval_k(e2, env, {
                        v2 => {
                            v1 match {
                                case Closure(x, funcBod, funcEnv) => {
                                    val newenv = Extend(x, v2, funcEnv)
                                    // Evaluate fucntion body with new environment
                                    eval_k(funcBod, newenv, k)
                                }
                               case _ => throw new IllegalArgumentException("Function call error: expression does not evaluate to a closure")
                            }
                        }
                    })
                }               
        })
    }
}

import scala.annotation.tailrec 


defined function eval_k

In [23]:
implicit def toExpr(d: Double) : Expr = Const(d)
implicit def toExpr(s: String): Expr = Ident(s)

def continue1(v: Value): String = v match {
    case NumValue(d) => d.toString
    case BoolValue(b) => b.toString
    case Closure(_,_,_) => "Some Closure"
}
def continue2(v: Value): Int = v match {
    case NumValue(_) => 1
    case BoolValue(_) => 2
    case Closure(_,_,_) => 3
}

val x = Ident("x")
val y = Ident("y")
val e1 = Plus(x, 1.0)
val e2 = Div(e1, Plus(1.0, y))
val e3 = Let("x", 1.0, e2)
val e4 = Let("y", 3.0, e3)

val v1: String = eval_k(e4, EmptyEnvironment, continue1)
assert(v1 == "0.5", s"Test 1: Expected 0.25 obtained: {$v1}")
// function(x) if (x >= 2) then (x+1)/(y+1) else (x+1)
val e5 = FunDef("x", IfThenElse(Geq(x, 2.0), e2, e1))
// let y = 4 in let f = e5 in f(y) // should return 1.0
val e6 = Let("y", 4, 
             Let("f", e5, 
                FunCall(Ident("f"), y)))
val v2 = eval_k(e6, EmptyEnvironment, continue1)
assert(v2 == "1.0", s"Test 2: Expected 1.0 obtained {$v2}")
passed(10)

Tests Passed (10 points)

defined function toExpr
defined function toExpr
defined function continue1
defined function continue2
x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
e1: Plus = Plus(e1 = Ident(s = "x"), e2 = Const(d = 1.0))
e2: Div = Div(
  e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 1.0)),
  e2 = Plus(e1 = Const(d = 1.0), e2 = Ident(s = "y"))
)
e3: Let = Let(
  id = "x",
  e1 = Const(d = 1.0),
  e2 = Div(
    e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 1.0)),
    e2 = Plus(e1 = Const(d = 1.0), e2 = Ident(s = "y"))
  )
)
e4: Let = Let(
  id = "y",
  e1 = Const(d = 3.0),
  e2 = Let(
    id = "x",
    e1 = Const(d = 1.0),
    e2 = Div(
      e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 1.0)),
      e2 = Plus(e1 = Const(d = 1.0), e2 = Ident(s = "y"))
    )
  )
)
v1: String = "0.5"
e5: FunDef = FunDef(
  param = "x",
  body = IfThenElse(
    eCond = Geq(e1 = Ident(s = "x"), e2 = Const(d = 2.0)),
    eThen = Div(
      e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 1.0)),
      e2 = Plus(e1 = Const

## Problem 2 : Error Handling Continuations (10 points)

We studied CPS style so far where each function in CPS form has one extra argument called the "continuation"

` fun_k(arg: ..., k: ResultType=> T) : T `


We will now have functions with two continuations: one continuation is the normal continuation we learned in class. The other continuation is called the "error continuation". It is called whenever the program encounters an error. Typically error continuation will take some arguments such as the error message, error type and so on to help the user debug the code. We will skip these and simply consider error continuations without arguments for simplicity. 

The type of our CPS function will become: 

` fun_k(arg: ..., k: ResultType=> T, err_k: Unit => T ) : T `

The idea is that if some error arises in the computation that would normally be handled by throwing an exception, we will call the error continuation instead.


In [24]:
def recursion_example1(x: Int): Int = {
    if  (x <= 0){
        1
    } else {
        if (x % 10 == 3){
            throw new IllegalArgumentException("I cannot handle 13 or any number that leaves a rem. of 3 when divided by 10")
        } else {
            val x1: Int = ((x-3)/2).toInt
            1+ recursion_example1(x1)
        }
    }
}

defined function recursion_example1

In [25]:
def recursion_example1_k[T](x: Int,  k: Int => T, err_k: () => T): T = {
    if (x <= 0){
        k(1)
    } else {
        if (x % 10 == 3){
            err_k() // Call the error continuation -- we will leave out arguments for simplicity
        } else {
            val x1: Int = ((x-3)/2).toInt
            recursion_example1_k(x1, v => { k(v+1)}, err_k)
        }
    }
}

defined function recursion_example1_k

In [26]:
println(recursion_example1(15))
println(recursion_example1_k(15, x => x, () => -1))

println(
    try {
      recursion_example1(29)
    } catch{
        case e:Exception => {println(s"Caught Exception $e")}
    }
   )
println(recursion_example1_k(29, x => x.toString, () => "Pericolo, Danger, Achtung!! Error Continuation got called."))

4
4
Caught Exception java.lang.IllegalArgumentException: I cannot handle 13 or any number that leaves a rem. of 3 when divided by 10
()
Pericolo, Danger, Achtung!! Error Continuation got called.


Instead of throwing a `RunTimeError` exception when you encounter an error, re-implement the interpreter to instead call an error continuation argument. You should reimplement the helper functions as well to take in error continuations.

__Warning__: We are going to redefine away the functions you wrote back in Problem 1 to raise an error. If you would like to go back to work on problem 1, please restart your kernel and run the cells again from beginning.

In [27]:
/* -- First make sure that we redfine the functions from previous problems to throw errors --*/


def addValue_k[T](v1: Value, v2: Value, k: Value => T): T = throw new IllegalArgumentException("Problem 2: addValue_k from problem 1  cannot be called")
def divValue_k[T](v1: Value, v2: Value, k: Value => T): T = throw new IllegalArgumentException("Problem 2: divValue_k from problem 1  cannot be called")
def geqValue_k[T](v1: Value, v2: Value, k: Value => T): T = throw new IllegalArgumentException("Problem 2: geqValue_k from problem 1 cannot be called")
def lookup_k[T](env: Environment, x: String, k: Value => T ) : T = throw new IllegalArgumentException("Problem 2: lookupValue_k from problem 1 cannot be called")

/* -- 
  Now let's implement things with error continuation 
  Implement the addValue helper function to add the two values and pass result to 
  continuation if they are nummbers.
  Otherwise, call the error continuation.
-- */
def addValueErr_k[T](v1: Value, v2: Value, k: Value => T, err_k: () => T ): T = (v1, v2) match {
    // YOUR CODE HERE
    case (NumValue(d1), NumValue(d2)) => {
        k(NumValue(d1+d2))
    }
    case _ => {
        err_k()
    }
}


/* Implement the function divValue using CPS style. However, if the second value v2 is zero, 
   then ensure that you call the error continuation. */
def divValueErr_k[T](v1: Value, v2: Value, k: Value => T, err_k: () => T ): T = (v1, v2) match {
    // YOUR CODE HERE
    case (NumValue(d1), NumValue(d2)) => {
        if (d2!=0) k(NumValue(d1/d2))
        else {
            err_k()
        }
    }
    case _ => {
        err_k()
    }   
}

// Same as above. Implement >= comparison of numbers but with error continuation called
// if there is an error.
def geqValueErr_k[T](v1: Value, v2: Value, k: Value => T, err_k: ()=> T ): T = (v1, v2) match {
    // YOUR CODE HERE
    case (NumValue(d1), NumValue(d2)) => {
        if (d1>=d2) {
            k(BoolValue(true))
        }
        else {
            k(BoolValue(false))
        }
    }
    case _ => {
        err_k()
    }
}

//Implement lookup of a value from the environment with continuation and error continuation.
//Error continuation will be called if there is an error.

def lookupErr_k[T](env: Environment, x: String, k: Value => T, err_k: ()=> T ): T = env match {
    // YOUR CODE HERE
    case EmptyEnvironment => err_k()
    case Extend(s, v, env2) if s == x => {
        k(v)
    }
    case Extend(s, v, env2)  => {
        lookupErr_k(env2, x, v1 => k(v1), err_k )
    }
}

/*-- 
Implement the eval function but now with the error continuation. 
--*/

def evalErr_k[T](e: Expr, env: Environment, k: Value => T, err_k: () => T ): T = e match {
    // YOUR CODE HERE
     case Const(f) => k(NumValue(f))

    case Ident(x) => {
        lookupErr_k(env,x,k,err_k)
    }
    
    case Plus(e1, e2) => {
       evalErr_k( e1, env, v1 => {
             evalErr_k(e2, env, v2 => {
                addValueErr_k(v1, v2, k,err_k)
          }, err_k)
       }, err_k)  
   }
    

    case Div(e1, e2) => {
        evalErr_k(e1,env, v1 => {
            evalErr_k(e2,env, v2 => {
                divValueErr_k(v1,v2,k,err_k)
            }, err_k)
        },err_k)
    }
    
    case Geq(e1, e2) => {
        evalErr_k(e1,env, v1 => {
            evalErr_k(e2,env, v2 => {
                geqValueErr_k(v1,v2,k,err_k)
            }, err_k)
        }, err_k)
    }

    case IfThenElse(e1, e2, e3) => {
        evalErr_k(e1, env, {
            case BoolValue(true) => evalErr_k(e2, env, k, err_k )
            case BoolValue(false) => evalErr_k(e3, env,k, err_k)
            case _ => err_k()
        }, err_k)
    }

    case Let(x, e1, e2) => {
        evalErr_k(e1, env, v1 => {
                val env2 = Extend(x, v1, env) // create a new extended env
                evalErr_k(e2, env2, k, err_k) // eval e2 under that.
            }, err_k)
    }

    case FunDef(x, e) => {
        k(Closure(x, e, env)) // Return a closure with the current enviroment.
    }

    case FunCall(e1, e2) => {
        evalErr_k(e1, env, {
                v1 => {
                    evalErr_k(e2, env, {
                        v2 => {
                            v1 match {
                                case Closure(x, funcBod, funcEnv) => {
                                    val newenv = Extend(x, v2, funcEnv)
                                    // Evaluate fucntion body with new environment
                                    evalErr_k(funcBod, newenv, k, err_k)
                                }
                               case _ => err_k()
                            }
                        }
                    }, err_k)
                }               
        }, err_k )
    }
    
}

defined function addValue_k
defined function divValue_k
defined function geqValue_k
defined function lookup_k
defined function addValueErr_k
defined function divValueErr_k
defined function geqValueErr_k
defined function lookupErr_k
defined function evalErr_k

In [28]:
implicit def toExpr(d: Double) : Expr = Const(d)
implicit def toExpr(s: String): Expr = Ident(s)


def continue1(v: Value): String = v match {
    case NumValue(d) => d.toString
    case BoolValue(b) => b.toString
    case Closure(_,_,_) => "Some Closure"
}
def continue2(v: Value): Int = v match {
    case NumValue(_) => 1
    case BoolValue(_) => 2
    case Closure(_,_,_) => 3
}

def errorHandler1() = {
    println("Successfully caught error in computation.")
    "errr"
}

// x + 10
val x = Ident("x")
val e1 = Plus(x, Const(10.0))


val v1 = evalErr_k(e1, EmptyEnvironment, continue1, errorHandler1)
assert (v1 == "errr", "Failed to catch error in interpreter for e1")


val e2 = Let("x", Const(5.0), e1)
val v2 = evalErr_k(e2, EmptyEnvironment, continue1, errorHandler1)
assert (v2 == "15.0", s"Test 2 failed: Expected 15.0 got {$v2}")

val e3 = Let("y", Const(5.0), e1)
val v3 = evalErr_k(e3, EmptyEnvironment, continue1, errorHandler1)
assert (v3 == "errr", s"Test 3 failed to catch error")

val e4 = Let("x", Div(Const(1.0), Const(0)), Const(10))
val v4 = evalErr_k(e3, EmptyEnvironment, continue1, errorHandler1)
assert (v4 == "errr", s"Test 4 failed to catch error")



val e5 = IfThenElse(Plus(5,10), Plus(10,15), Plus(15,20) )
val v5 = evalErr_k(e5, EmptyEnvironment, continue1, errorHandler1)
assert (v5 == "errr", s"Test 5 failed to catch error")

val e6 = IfThenElse(Geq(Plus(5,10), Plus(10,5)), Plus(10,15), Plus(15,20) )
val v6 = evalErr_k(e6, EmptyEnvironment, continue1, errorHandler1)
assert (v6 == "25.0", s"Test 6 Expected: 25.0 obtained $v6")

val e7 = FunCall(e6, Const(6.0))
val v7 = evalErr_k(e7, EmptyEnvironment, continue1, errorHandler1)
assert (v7 == "errr", s"Test 7: Failed to catch error")

val e8 = Geq(Geq(10, 5), 4)
val v8 = evalErr_k(e8, EmptyEnvironment, continue1, errorHandler1)
assert (v8 == "errr", s"Test 8: Failed to catch error")


val e9 = Plus(Geq(10, 5), 4)
val v9 = evalErr_k(e9, EmptyEnvironment, continue1, errorHandler1)
assert (v9 == "errr", s"Test 9: Failed to catch error")


val e10 = Div(5, Geq(10, 5))
val v10 = evalErr_k(e10, EmptyEnvironment, continue1, errorHandler1)
assert (v10 == "errr", s"Test 10: Failed to catch error")

passed(15)


Successfully caught error in computation.
Successfully caught error in computation.
Successfully caught error in computation.
Successfully caught error in computation.
Successfully caught error in computation.
Successfully caught error in computation.
Successfully caught error in computation.
Successfully caught error in computation.
Tests Passed (15 points)

defined function toExpr
defined function toExpr
defined function continue1
defined function continue2
defined function errorHandler1
x: Ident = Ident(s = "x")
e1: Plus = Plus(e1 = Ident(s = "x"), e2 = Const(d = 10.0))
v1: String = "errr"
e2: Let = Let(
  id = "x",
  e1 = Const(d = 5.0),
  e2 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 10.0))
)
v2: String = "15.0"
e3: Let = Let(
  id = "y",
  e1 = Const(d = 5.0),
  e2 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 10.0))
)
v3: String = "errr"
e4: Let = Let(
  id = "x",
  e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
  e2 = Const(d = 10.0)
)
v4: String = "errr"
e5: IfThenElse = IfThenElse(
  eCond = Plus(e1 = Const(d = 5.0), e2 = Const(d = 10.0)),
  eThen = Plus(e1 = Const(d = 10.0), e2 = Const(d = 15.0)),
  eElse = Plus(e1 = Const(d = 15.0), e2 = Const(d = 20.0))
)
v5: String = "errr"
e6: IfThenElse = IfThenElse(
  eCond = Geq(
    e1 = Plus(e1 = Const(d = 5.0), e2 = Const(d = 10.0)),
    e2 = Plus(e1 = Const(d = 10.0), e2 = Const(d = 

## Problem 3: Try-Catch In Lettuce (25 points)

__Warning:__ Please do not attempt this problem before finishing problem 2. You will be asked to cut and paste code from problem 2 in here. This is in general bad practice but we are doing so to simplify the jupyter notebook and make testing easy. 

We will now extend the grammar of Lettuce to add `try-catch` block to handle exceptions.

$$\begin{array}{rcll}
  \mathbf{Expr} & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Div}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Geq}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{IfThenElse}(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{FunDef}(\mathbf{Ident}, \mathbf{Expr}) \\ 
  & | & \text{FunCall}(\mathbf{Expr}, \mathbf{Expr}) \\ 
  & | & \color{red}{\text{TryCatch}(\mathbf{Expr}, \mathbf{Expr})} \\ 
  \end{array}$$
  
 In terms of concrete syntax, let us illustrate with examples: 
 
 Consider an expression 
 ~~~
 try 
   1/0
 catch 
    2
 ~~~
 
 The evaluation of `1/0` throws an error which will be "caught" by the try/catch block wrapping our expression and the overall expression will evaluate to `2`.
 
Here is another example

~~~
let z = 10 in 
  try 
    let f = function (x) y + x in 
      f(10) 
   catch 
     (25 + z)
~~~

The call to `f(10)` will lead to an exception since `y` is not defined. This will be caught  and the result will be `25 + 10`.

Try-Catch blocks can be nested: 

~~~
  try 
     (try 1/0 catch 10)  + (try 2/0 catch -10)
   catch 
     25
~~~

The entire expression will evaluate to `0`. 
  - The first argument to `+` itself is wrapped around a try catch block that will evaluate to `10`.
  - The second block will evaluate to `-10` due to another exception that is thrown when evaluating `2/0`. 

Just like in other programming languages, the innermost catch block is the one that will handle the exception.

Consider the example below:
~~~
try 
 try 
   10/0
 catch 25
catch 15
~~~

This will evaluate to 25.

Exceptions can be thrown by the expression inside a catch block too.

For example,

~~~
try 
   10/0
catch 
    x+ y
~~~

will lead to an error.

However, if we had:


~~~
try 
  try 
     10/0
  catch 
      x+ y
catch 
   10
~~~

This will evaluate to `10`.





$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\; (\mathsf{#3})} $$

If the first argument of `TryCatch` evaluates without error, the entire expression evaluates to the same value as the first argument.

$$\semRule{ \textsf{eval}(\texttt{e1}, env) = v_1,\ v_1 \not= \mathbf{error}}{
\textsf{eval}(\texttt{TryCatch(e1, e2)}, env) = v_1 }{try-catch-ok} $$

On the other hand, if the first argument `e1` evaluates to an error, the entire expression evaluates to whatever the second argument evaluates to.
$$\semRule{ \textsf{eval}(\texttt{e1}, env) =\mathbf{error}, \textsf{eval}(\texttt{e2}, env) = v_2}{
\textsf{eval}(\texttt{TryCatch(e1, e2)}, env) = v_2 }{try-catch-ok} $$


The goal of this problem is to evaluate Lettuce with Try/Catch exception handling but without using Scala's exception handling mechanism (that would be cheating :-)
Instead, we will use error continuations to handle try/catch blocks.

__Note__: The code for this problem should reuse functions that were defined for the previous problem. 

In [29]:
case class TryCatch(e1: Expr, e2: Expr) extends Expr

defined class TryCatch

In [31]:
def evalErr_k[T](e: Expr, 
                 env: Environment, 
                 k: Value => T, 
                 err_k: () => T ): T = e match {
   /* -- Cut and Paste from Solution to Previous Problem for all cases --*/
   case Const(d) => k(NumValue(d))

    case Ident(x) => {
        lookupErr_k(env,x,k,err_k)
    }
    
    case Plus(e1, e2) => {
       evalErr_k( e1, env, v1 => {
             evalErr_k(e2, env, v2 => {
                addValueErr_k(v1, v2, k,err_k)
          }, err_k)
       }, err_k)  
   }
    

    case Div(e1, e2) => {
        evalErr_k(e1,env, v1 => {
            evalErr_k(e2,env, v2 => {
                divValueErr_k(v1,v2,k,err_k)
            }, err_k)
        },err_k)
    }
    
    case Geq(e1, e2) => {
        evalErr_k(e1,env, v1 => {
            evalErr_k(e2,env, v2 => {
                geqValueErr_k(v1,v2,k,err_k)
            }, err_k)
        }, err_k)
    }

    case IfThenElse(e1, e2, e3) => {
        evalErr_k(e1, env, {
            case BoolValue(true) => evalErr_k(e2, env, k, err_k )
            case BoolValue(false) => evalErr_k(e3, env,k, err_k)
            case _ => err_k()
        }, err_k)
    }

    case Let(x, e1, e2) => {
        evalErr_k(e1, env, v1 => {
                val env2 = Extend(x, v1, env) // create a new extended env
                evalErr_k(e2, env2, k, err_k) // eval e2 under that.
            }, err_k)
    }

    case FunDef(x, e) => {
        k(Closure(x, e, env)) // Return a closure with the current enviroment.
    }

    case FunCall(e1, e2) => {
        evalErr_k(e1, env, {
                v1 => {
                    evalErr_k(e2, env, {
                        v2 => {
                            v1 match {
                                case Closure(x, funcBod, funcEnv) => {
                                    val newenv = Extend(x, v2, funcEnv)
                                    // Evaluate fucntion body with new environment
                                    evalErr_k(funcBod, newenv, k, err_k)
                                }
                               case _ => err_k()
                            }
                        }
                    }, err_k)
                }               
        }, err_k )
    }
   
   /*-- Complete code for try-catch block --*/
    case TryCatch(e1, e2) => {
        evalErr_k(e1, env, k, () => { //if e1 is error we eval e2
            evalErr_k(e2, env, k, err_k )
        })
    }
}

defined function evalErr_k

In [32]:

implicit def toExpr(d: Double) : Expr = Const(d)
implicit def toExpr(s: String): Expr = Ident(s)


def continue1(v: Value): Double= v match {
    case NumValue(d) => d
    case BoolValue(b) => if (b) 1.0 else -1.0
    case Closure(_,_,_) => 1000.111
}
def continue2(v: Value): Int = v match {
    case NumValue(_) => 1
    case BoolValue(_) => 2
    case Closure(_,_,_) => 3
}

def errorHandler1() = {
    println("Successfully caught error in computation.")
    "error"
}

// try 1/0 catch 10 (division by zero err handle)
val e1 = TryCatch(Div(1,0), 10)
val v1 = evalErr_k(e1, EmptyEnvironment, continue1, errorHandler1)
println(s"v1 = {$v1}")
assert(v1 == 10.0, "Test 1 failed")
 
// try x + 5 catch -25 (unknown identifier err handle)
val e2 = TryCatch(Plus(x, 5), -25)
val v2 = evalErr_k(e2, EmptyEnvironment, continue1, errorHandler1)
println(s"v2 = {$v2}")
assert(v2 == -25.0, "Test 2 failed")

//try if (5+10) then 5 else 4 catch 15 (if-then-else err handle)
val e3 = TryCatch( IfThenElse(Plus(5,10), 5, 4), 15.0)
val v3 = evalErr_k(e3, EmptyEnvironment, continue1, errorHandler1)
println(s"v3 = {$v3}")
assert(v3 == 15.0, "Test 3 failed")


//try let f = 25 in f(25) catch -5 (function call type mismatch)

val e4 = TryCatch(Let("f", 25, FunCall("f", 25)), -5)
val v4 = evalErr_k(e4, EmptyEnvironment, continue1, errorHandler1)
println(s"v4 = {$v4}")
assert(v4 == -5.0, "Test 4 failed")

//try  25 + (10 >= 5) in  catch -5 (function call type mismatch)
val e5 = TryCatch(Plus(25, Geq(10, 5)), Geq(10, 5))
val v5 = evalErr_k(e5, EmptyEnvironment, continue1, errorHandler1)
println(s"v5 = {$v5}")
assert(v5 == 1.0, "Test 4 failed")


passed(10)


v1 = {10.0}
v2 = {-25.0}
v3 = {15.0}
v4 = {-5.0}
v5 = {1.0}
Tests Passed (10 points)

defined function toExpr
defined function toExpr
defined function continue1
defined function continue2
defined function errorHandler1
e1: TryCatch = TryCatch(
  e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
  e2 = Const(d = 10.0)
)
v1: Any = 10.0
e2: TryCatch = TryCatch(
  e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 5.0)),
  e2 = Const(d = -25.0)
)
v2: Any = -25.0
e3: TryCatch = TryCatch(
  e1 = IfThenElse(
    eCond = Plus(e1 = Const(d = 5.0), e2 = Const(d = 10.0)),
    eThen = Const(d = 5.0),
    eElse = Const(d = 4.0)
  ),
  e2 = Const(d = 15.0)
)
v3: Any = 15.0
e4: TryCatch = TryCatch(
  e1 = Let(
    id = "f",
    e1 = Const(d = 25.0),
    e2 = FunCall(e1 = Ident(s = "f"), e2 = Const(d = 25.0))
  ),
  e2 = Const(d = -5.0)
)
v4: Any = -5.0
e5: TryCatch = TryCatch(
  e1 = Plus(
    e1 = Const(d = 25.0),
    e2 = Geq(e1 = Const(d = 10.0), e2 = Const(d = 5.0))
  ),
  e2 = Geq(e1 = Const(d = 10.0), e2 = Const(d = 5.0))
)
v5: Any = 1.0

In [33]:
//Test nesting of try catch blocks.


implicit def toExpr(d: Double) : Expr = Const(d)
implicit def toExpr(s: String): Expr = Ident(s)


def continue1(v: Value): Double= v match {
    case NumValue(d) => d
    case BoolValue(b) => if (b) 1.0 else -1.0
    case Closure(_,_,_) => 1000.111
}
def continue2(v: Value): Int = v match {
    case NumValue(_) => 1
    case BoolValue(_) => 2
    case Closure(_,_,_) => 3
}

def errorHandler1() = {
    println("Successfully caught error in computation.")
    "error"
}


/* 
 try 
    let x = try 1/0 catch 10 in 
      x/(x+(-10))
  catch 
    15
*/


val e11 = TryCatch(Div(1,0), 10)
val e12 = Let("x", e11, Div("x", Plus("x", -10)))
val e1 = TryCatch(e12, 15)
val v1 = evalErr_k(e1, EmptyEnvironment, continue1, errorHandler1)
println(s"v1 = {$v1}")
assert (v1 == 15.0, "Test 1 failed")

/* 
 try 
    let x = try 1/0 catch 10 in 
       x + 10
  catch 
    15
*/

val e21 = TryCatch(Div(1,0), 10)
val e22 = Let("x", e21, Plus("x", 10))
val e2 = TryCatch(e22, 15)
val v2 = evalErr_k(e2, EmptyEnvironment, continue1, errorHandler1)
println(s"v2 = {$v2}")
assert (v2 == 20.0, "Test 2 failed")



/* 
 try 
    try 
      try 
         x + 10
      catch 
        25
    catch 
      20
  catch 
    15
*/

val e3 = TryCatch( TryCatch(TryCatch(Plus(x, 10), 25), 20), 15)
val v3 = evalErr_k(e3, EmptyEnvironment, continue1, errorHandler1)
println(s"v3 = {$v3}")
assert (v3 == 25.0, "Test 3 failed")


/*
let z = 10 in 
  try 
    let f = function (x) y + x in 
      f(10) 
   catch 
     (25 + z)
*/

val e41= FunDef("x", Plus("y", "x"))
val e42 =Let("f", e41, FunCall("f", 10.0))
val e43 = TryCatch(e42, Plus(25, "z"))
val e4 = Let("z", 10.0, e43)

val v4 = evalErr_k(e4, EmptyEnvironment, continue1, errorHandler1)
println(s"v4 = {$v4}")
assert (v4 == 35.0, "Test 4 failed")

passed(10)

v1 = {15.0}
v2 = {20.0}
v3 = {25.0}
v4 = {35.0}
Tests Passed (10 points)

defined function toExpr
defined function toExpr
defined function continue1
defined function continue2
defined function errorHandler1
e11: TryCatch = TryCatch(
  e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
  e2 = Const(d = 10.0)
)
e12: Let = Let(
  id = "x",
  e1 = TryCatch(
    e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
    e2 = Const(d = 10.0)
  ),
  e2 = Div(
    e1 = Ident(s = "x"),
    e2 = Plus(e1 = Ident(s = "x"), e2 = Const(d = -10.0))
  )
)
e1: TryCatch = TryCatch(
  e1 = Let(
    id = "x",
    e1 = TryCatch(
      e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
      e2 = Const(d = 10.0)
    ),
    e2 = Div(
      e1 = Ident(s = "x"),
      e2 = Plus(e1 = Ident(s = "x"), e2 = Const(d = -10.0))
    )
  ),
  e2 = Const(d = 15.0)
)
v1: Any = 15.0
e21: TryCatch = TryCatch(
  e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
  e2 = Const(d = 10.0)
)
e22: Let = Let(
  id = "x",
  e1 = TryCatch(
    e1 = Div(e1 = Const(d = 1.0), e2 = Const(d = 0.0)),
    e2 = C

In [34]:

implicit def toExpr(d: Double) : Expr = Const(d)
implicit def toExpr(s: String): Expr = Ident(s)

def continue1(v: Value): Double= v match {
    case NumValue(d) => d
    case BoolValue(b) => if (b) 1.0 else -1.0
    case Closure(_,_,_) => 1000.111
}
def continue2(v: Value): Int = v match {
    case NumValue(_) => 1
    case BoolValue(_) => 2
    case Closure(_,_,_) => 3
}

def errorHandler1() = {
    println("Successfully caught error in computation.")
    "error"
}

/* 
      try 
         x + 10
      catch 
        25 + y
*/
val e = TryCatch(Plus("x", 10), Plus(25, "y"))
val v = evalErr_k(e, EmptyEnvironment, continue1, errorHandler1)
println(s"v = $v")
assert(v == "error", "Test 1 Failed")


/* 
  try
      try 
         x + 10
      catch 
        25 + y
   catch 
      15
*/

val e2 = TryCatch(e, 15.0)
val v2 = evalErr_k(e2, EmptyEnvironment, continue1, errorHandler1)
println(s"v2 = $v2")
assert(v2 == 15.0, "Test 2 Failed")


/* 
  try
      try 
         x + 10
      catch 
        (
         try 25 + y catch -5 
        )
   catch 
      15
*/

val e3 = TryCatch( TryCatch(
                      Plus("x", 10), 
                      TryCatch(Plus(25, "y"), -5))
                  , 15)
val v3 = evalErr_k(e3, EmptyEnvironment, continue1, errorHandler1)
println(s"v3 = $v3")
assert(v3 == -5.0, "Test 3 Failed")


passed(5)

Successfully caught error in computation.
v = error
v2 = 15.0
v3 = -5.0
Tests Passed (5 points)

defined function toExpr
defined function toExpr
defined function continue1
defined function continue2
defined function errorHandler1
e: TryCatch = TryCatch(
  e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 10.0)),
  e2 = Plus(e1 = Const(d = 25.0), e2 = Ident(s = "y"))
)
v: Any = "error"
e2: TryCatch = TryCatch(
  e1 = TryCatch(
    e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 10.0)),
    e2 = Plus(e1 = Const(d = 25.0), e2 = Ident(s = "y"))
  ),
  e2 = Const(d = 15.0)
)
v2: Any = 15.0
e3: TryCatch = TryCatch(
  e1 = TryCatch(
    e1 = Plus(e1 = Ident(s = "x"), e2 = Const(d = 10.0)),
    e2 = TryCatch(
      e1 = Plus(e1 = Const(d = 25.0), e2 = Ident(s = "y")),
      e2 = Const(d = -5.0)
    )
  ),
  e2 = Const(d = 15.0)
)
v3: Any = -5.0

### That's all Folks!!